# Week 7: Everything in Code
Today I will show you how to write up OLS estimation in Julia code.

I'm choosing Julia because it let's me implement everything using linear algebra. So instead of resorting to a canned command, such as Stata's "regress Y X" the point here is to be explicit about what's going on under the hood.

When you type "regress Y X" in Stata then magically the results appear. But where do they come from? Under the hood Stata is doing what we will be doing here by using Julia.

I will present three different Jupyter notebooks to you, at increasing level of abstraction and complexity. 

* ols_1: a primitive OLS implementation to expose you to the main idea

* ols_2: does exactly the same thing as ols_1, but slightly more abstract to prepare you for ols_3

* ols_3: highest level of abstraction and highest degree of flexibility, allows us to do Monte Carlo simulations.

In all three codes I'm doing something funny: I'm creating my own data!

In real life you usually deal with one random sample that is given to you. For example, you observe ordered pairs of earnings and schooling and, in Stata, you would do something like "regress earnings schooling" to estimate the projection coefficient in the linear projection of earnings on schooling. The underlying model for this estimation is

$$
earnings = \beta_0 + \beta_1 schooling + e, \qquad E(e \cdot schooling) = 0
$$

My interpretation of this model is this: schooling and $e$ are random variables that generate the random variable earnings according to the linear model $\beta_0 + \beta_1 schooling + e$. The random sample consists of $N$ realizations of the ordered pairs (schooling, earnings) and my goal is to estimate $\beta_1$.

When I create my own random sample, here is what I do: I generate $N$ realizations for the random variables $X$ and $e$ (using statistical distributions of my own choosing) and then generate $Y$ based on $\beta_0 + \beta_1 X + e$. For this I need to "know" these things:
 
* the statistical distributions of $X$ and $e$

* the coefficients $\beta_0$ and $\beta_1$

(I don't "know" these. Instead I "create" or "choose" these. I'm assuming the role of the oracle!)

Then I pretend I only observe $N$ realizations of ordered pairs $(X,Y)$ and estimate $\beta_1$.

Why do I estimate $\beta_1$ when I created it in the first place? It gives me the opportunity to evaluate numerically if a particular estimation method is good. For example, if I set $\beta=5$ and I estimate $\hat{\beta}=42$ then I suspect that my estimator is not very good. You will hopefully understand the idea of these simulation exercises once we have worked through all three OLS notebooks. Bear with me.

# Data Generating Process

In this section I am creating the sample data according to a *true* model of my creation. In other words, I am "creating" or "choosing" the true model.

Consider the simple one equation model

\begin{align}
Y &= \beta_0 + \beta_1 X + e
\end{align}


All variables are scalars for simplicity. 

The variable $X$ is generated given the following information:

* $X \sim N(0,1)$

* $e \sim N(0,1)$
    
* $\beta_0 = 24$ and $\beta_1=8$ (the particular values here are irrelevant, I picked these for no good reason)

* sample size $N=1000$

Note that I could have chosen any statistical distribution for $X$ and $e$, they also don't have to have the same distribution of course. Also notice that I effectively create $X$ to be statistically independent of $e$, so the linear projection model is adequate. No endogeneity here. Yet.

# Implementation in Julia

In [6]:
# loading modules
using LinearAlgebra
using Statistics
using Random

# Setting parameters and creating sample

In [7]:
# setting parameters
N = 1000 # sample size
beta_0 = 24 # intercept
beta_1 = 8 # slope

# creating sample
Random.seed!(42) # fixing random numbers for cross-code comparison

# exogenous variables: simply set as normal rvs
X = [ones(N, 1) randn(N)]
e = randn(N) 
    
# creating endogenous variables    
Y = X*[beta_0; beta_1] + e;

# Estimation (OLS estimator, Avar, se, ci)
Remember the following results from our lecture:

* $\hat{\beta} = (X'X)^{-1} X'Y$

* $\sqrt{N} (\hat{\beta}-\beta) \to_d N(0,\Omega)$ where $\Omega \in \{\Omega_{hom}, \Omega_{het} \}$

    * $\Omega_{hom} = \sigma_e^2 E(X_i X_i')^{-1}$
    
    * $\Omega_{het} = E(X_i X_i')^{-1} E(e_i^2 X_i X_i') E(X_i X_i')^{-1}$
    
The practical meaning of the asymptotic distribution is this:

* $\hat{\beta} \overset{approx}{\sim} N(\beta, \hat{\Omega}/N)$ where

    * $\hat{\Omega}_{hom} := s_e^2 \cdot (X'X/N)^{-1}$
    
    * $\hat{\Omega}_{het} := (X'X/N)^{-1} \cdot (X' diag(\hat{e}^2) X/N) \cdot (X'X/N)^{-1}$
    
    * $s_e^2 := \hat{e}' \hat{e}/N$

In [8]:
# OLS estimation

bols = inv(X'*X)*X'*Y;
println("OLS estimator:  ", bols)

OLS estimator:  [24.015845699885638, 8.047945398812304]


In [9]:
# Asymptotic variance estimation

ehat = Y - X*bols;
shat = ehat'*ehat/length(ehat);

Omegahat_hom = shat[1]*inv(X'*X);
Omegahat_het = inv(X'*X) * (X'*Diagonal(ehat.^2)*X) * inv(X'*X);

se_hom = diag(Omegahat_hom).^(0.5);
se_het = diag(Omegahat_het).^(0.5);

println("se homosk.:   ", se_hom)
println("se hetersk.:  ", se_het)

se homosk.:   [0.031154794344993554, 0.030061293182925403]
se hetersk.:  [0.031240701084897, 0.02878034246262317]


In [10]:
# Confidence interval

ci_hom = [bols[2] - 1.96*se_hom[2]; bols[2] + 1.96*se_hom[2]];
ci_het = [bols[2] - 1.96*se_het[2]; bols[2] + 1.96*se_het[2]];
println("conf interval homosk:   ", ci_hom)
println("conf interval heterosk: ", ci_het)

conf interval homosk:   [7.989025264173771, 8.106865533450838]
conf interval heterosk: [7.991535927585563, 8.104354870039046]
